# Dekoratory w praktyce

Od prostych opakowań po dekoratory parametryzowane i klasowe.


## Cele
- przypomnieć mechanikę dekoratorów i znaczenie `functools.wraps`
- zaprezentować dekoratory z argumentami oraz klasowe
- pokazać kompozycję dekoratorów w realnych problemach


## Przypomnienie mechanizmu
Dekorator to funkcja przyjmująca funkcję i zwracająca funkcję.
Najpierw tworzymy prosty przykład mierzący czas wykonania.


In [ ]:
import time
from functools import wraps

def measure_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = (time.perf_counter() - start) * 1000
        print(f"{func.__name__} -> {elapsed:.2f} ms")
        return result
    return wrapper

@measure_time
def compute():
    # przykładowa praca CPU
    return sum(range(1_000))

compute()


## Dekoratory z argumentami
Dodajemy poziom pośredni przyjmujący konfigurację, np. próg logowania.


In [ ]:
from functools import wraps

def notify(threshold_ms: float):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start = time.perf_counter()
            result = func(*args, **kwargs)
            elapsed = (time.perf_counter() - start) * 1000
            if elapsed > threshold_ms:
                print(f"[ALERT] {func.__name__} trwała {elapsed:.2f} ms")
            return result
        return wrapper
    return decorator

@notify(0.1)
def no_op():
    return sum(range(1000))

no_op()


## Dekoratory klasowe
Klasa implementująca `__call__` może stać się dekoratorem.
Pozwala przechowywać stan między wywołaniami.


In [ ]:
class CountCalls:
    def __init__(self, func):
        wraps(func)(self)
        self.func = func
        self.calls = 0

    def __call__(self, *args, **kwargs):
        self.calls += 1
        print(f"{self.__name__} wywołana {self.calls} raz(y)")
        return self.func(*args, **kwargs)

@CountCalls
def greet(name: str):
    print(f"Cześć, {name}!")

for person in ["Ada", "Beata"]:
    greet(person)


## Dekoratory z biblioteki standardowej
`functools` oferuje gotowe rozwiązania: `lru_cache`, `singledispatch`, `cached_property`.


In [ ]:
from functools import lru_cache, singledispatch

@lru_cache(maxsize=None)
def fibonacci(n: int) -> int:
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print(fibonacci(10))

@singledispatch
def render(value):
    raise TypeError("Nieobsługiwany typ")

@render.register
def _(value: int):
    return f"Liczba: {value}"

@render.register
def _(value: list):
    return f"Lista o długości {len(value)}"

print(render(5))
print(render([1, 2, 3]))


**Podsumowanie:** Dekoratory pozwalają rozszerzać funkcje i metody bez ingerencji w ich kod.

**Pytanie kontrolne:** Po co stosować `functools.wraps` w dekoratorach?


### 🧩 Zadanie 1
Napisz dekorator `retry`, który ponowi wykonanie funkcji określoną liczbę razy,
zanim zgłosi wyjątek. Liczbę powtórzeń przyjmij jako argument dekoratora.


In [ ]:
# Rozwiązanie Zadania 1
from functools import wraps

def retry(attempts: int):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            last_exc = None
            for _ in range(attempts):
                try:
                    return func(*args, **kwargs)
                except Exception as exc:  # łapiemy wyjątek i zapisujemy
                    last_exc = exc
            raise last_exc
        return wrapper
    return decorator

counter = {"calls": 0}

@retry(3)
def flaky():
    counter["calls"] += 1
    if counter["calls"] < 3:
        raise RuntimeError("Jeszcze nie!")
    return "Sukces"

print(flaky())


### 🧩 Zadanie 2
Przygotuj dekorator klasowy `@audit`, który każdą publiczną metodę
opakuje prostym logowaniem wejścia i wyjścia.


In [ ]:
# Rozwiązanie Zadania 2
import inspect
from functools import wraps

def audit(cls):
    for name, member in inspect.getmembers(cls, inspect.isfunction):
        if name.startswith("_"):
            continue  # pomijamy metody prywatne

        @wraps(member)
        def wrapper(self, *args, __orig=member, **kwargs):
            print(f"[AUDIT] {cls.__name__}.{__orig.__name__} args={args} kwargs={kwargs}")
            result = __orig(self, *args, **kwargs)
            print(f"[AUDIT] wynik={result}")
            return result
        setattr(cls, name, wrapper)
    return cls

@audit
class Calculator:
    def add(self, a, b):
        return a + b

    def sub(self, a, b):
        return a - b

calc = Calculator()
calc.add(2, 3)
